## Problem Statment

You have been provided a dataset that consists of echo-location clicks of two types of whales, namely, Gervais and Cuviers. In this assignment, your task is to classify the different types whales using Gradient Boosting with the help of the XGBoost library. You are expected to fill in functions that would complete this task. We use XGBoost here instead of GradientBoostedTrees in Spark because XGBoost running on a single machine is much faster than Spark running on 10 machines.

The data files were preprocessed on PySpark (10 nodes) cluster. The code for the same can be found in Data_Processing_Whales.ipynb. The preprocessed data is a numpy array with `4175` rows (for the 10mb file) with following columns (zero-indexed):
* Col 0-9: projections on first 10 eigen vectors
* Col 10: rmse
* Col 11: peak2peak
* Col 12: label (`0 if row.species==u'Gervais' else 1`)

You can also refer to XGBoost_Whales.ipynb under for more details on the XGBoost Analysis before you attempt this assignment.

Both Data_Processing_Whales.ipynb and XGBoost_Whales.ipynb can be found under XGBoost directory that was uploaded in edX as a part of "Notebooks for weeks 7 & 8". 

## XGBoost - Theory

A brief overview of gradient boosting in XGBoost can be found here:

* http://xgboost.readthedocs.io/en/latest/model.html
* http://xgboost.readthedocs.io/en/latest/python/python_intro.html



Use the XGBoost API for training and predicting scores: 

* http://xgboost.readthedocs.io/en/latest/python/python_api.html

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
      * **verbose_eval** - This can be used to control how much information the train function prints. You might want to set to **False** to avoid printing logs.
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      * **output_margin** - Whether to output the raw untransformed margin value (Use: output_margin=True)

## Notebook Setup

### Importing Required Libraries

In [1]:
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import random

### Loading Data

In [2]:
with open('Data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('Data/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('Data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### Setting Parameters for XG Boost
* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations

[Full description of options](https://xgboost.readthedocs.io/en/latest//parameter.html)

In [3]:
#You can change this cell if you wish to, but you aren't expected to
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 1     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'

    plst = param.items()
    return plst

## Exercises

### Computing the score ranges

The function <font color="blue">calc_stats</font> takes the xgboost margin scores as input and returns two numpy arrays min_scr, max_scr which are calculated as follows:

1. **min_scr**: mean - (3 $\times$ std)
2. **max_scr**: mean + (3 $\times$ std)

Here the input margin scores, represents the processed XGBoost margin scores obtained from the <font color="blue">bootstrap_pred</font> function. Each row represents the various scores for a specific example in an iteration and your <font color="blue">calc_stats</font> function is supposed to compute the **min_scr** and **max_scr** as defined for each example. So in the example below, we take a scenario where we have 3 examples which have 4 values each (From 4 bootstrap iterations).


**<font color="magenta" size=2>Example Input</font>**
``` python
[[-0.22 -0.19 -0.17 -0.13][-0.1 -0.05 0.02 0.10][0.03 0.11 0.12 0.15]]
```

Output: min_scr (numpy array), max_scr (numpy array)

**<font color="blue" size=2>Example Output</font>**
``` python
(array([-0.28 -0.23 -0.03]),
 array([-0.08  0.22  0.24]))
```

**Note**: Ensure you round the values in the numpy arrays to two decimal places

In [5]:
def calc_stats(margin_scores):
    min_scr = []
    max_scr = []
    # calculate margin scores
    for i in margin_scores:
        min_scr.append(np.mean(i)-(3*np.std(i)))  # mean - 3 * std dev
        max_scr.append(np.mean(i)+(3*np.std(i)))  # mean - 3 * std dev
        
    return (np.round(np.array(min_scr), decimals=2),
            np.round(np.array(max_scr), decimals=2)) # return lists rounded 2 decimals


In [6]:
margin_score = np.array([[-0.22, -0.19, -0.17, -0.13], [-0.1, -0.05, 0.02, 0.10], [0.03, 0.11, 0.12, 0.15]])
min_score, max_score = calc_stats(margin_score)
assert type(min_score) == np.ndarray, 'Incorrect Return type'
assert type(max_score) == np.ndarray, 'Incorrect Return type'

In [7]:
assert (min_score == np.array([-0.28, -0.23, -0.03])).all(), "Incorrect return value"

In [8]:
assert (max_score == np.array([-0.08,  0.22,  0.24])).all(), "Incorrect return value"

In [9]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [10]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


### Calculating predictions

Based on the ranges for each of the examples, i.e, (min_scr, max_scr), we can predict whether it's a Gervais or a Cuvier. Since all our scores will be between -1 and +1, we use 0 as the margin line. All examples which are on the left side of the margin, can be classified as Cuvier's and all which are on the right side can be classified as Gervais. However, since we take margin scores from a set of bootstraps for each example, we use the minimum and maximum score arrays to predict to determine the classification.

The function <font color="blue">predict</font> takes the minimum score array and maximum score array and returns predictions as follows:

1. If respective minimum score and maximum score values are less than 0, predict -1 (**Cuvier's**)
2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict 0 (**Unsure**)
3. If respective minimum score and maximum score values are greater than 0, predict 1 (**Gervais**)

**<font color="magenta" size=2>Example Input</font>**
``` python
min_scr (numpy array) = [-0.78 -0.68 -0.6 -0.53 -0.47 -0.42 -0.32 -0.21 -0.07 0.22]

max_scr (numpy array) = [-0.49 -0.39 -0.33 -0.25 -0.2 -0.11 -0.04 0.1 0.3 0.51]
```
Output: pred (numpy array of predictions)

**<font color="blue" size=2>Example Output</font>**
``` python
[-1 -1 -1 -1 -1 -1 -1  0  0  1]
```

In [24]:
def predict(min_scr, max_scr):
        pred = []
        for i in range(len(min_scr)):
            # if min_scr and max_scr < 0 then Cuvier's
            if min_scr[i] < 0 and max_scr[i] < 0:
                pred.append(-1)
            
            # If min_scr and max_scr > 0 then Gervais
            elif min_scr[i] > 0 and max_scr[i] > 0:
                pred.append(1)
                
            # If min_scr < 0 and max_scr > 0 then uncertain
            else:
                pred.append(0)
                
        return np.array(pred)


In [25]:
max_s = np.array([-0.49, -0.39, -0.33, -0.25, -0.2, -0.11, -0.04, 0.1, 0.3, 0.51])
min_s = np.array([-0.78, -0.68, -0.6, -0.53, -0.47, -0.42, -0.32, -0.21, -0.07, 0.22])
pred = predict(min_s, max_s)
true_pred = np.array([-1, -1, -1, -1, -1, -1, -1, 0, 0, 1])


In [26]:
assert type(pred) == np.ndarray, 'Incorrect return type'

In [27]:
assert (pred == true_pred).all(), 'Incorrect return value'

### Calculating scores

The function <font color="blue">bootstrap_pred</font> takes as input:

1. **Training set**
1. **Test set**
1. **n_bootstrap** Number of bootstrap samples that run XGBoost and trains one part of the sample set.
1. **minR, maxR** two numbers such that $0 < minR < maxR < 1$ that define the fractions of the `n_bootstrap` scores that define the range.
1. **bootstrap_size** - Number of bootstrap samples on which you will run XGBoost.
1. **num_round** - Number of iterations for running xgboost
1. **plst** - Parameter List

The output should be a confidence interval for each example in the test set. Together with a prediction that is `Gervais / Cuviers / Unsure`. The prediction `unsure` is to be output if the confidence interval contains the point 0.
After generating the confidence intervals, the function <font color="blue">predict</font> can be used to make predictions.

**Procedure**

Repeat the given procedure for n_bootstrap number of iterations:

For **n_bootstrap** iterations:
* Sample **boostrap_size** indices from the training set **with replacmennt**
* Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
* Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API. Ensure you set **output_margin=True** to get raw untransformed output scores.
* Normalize them by dividing them with the normalizing factor as max(scores) - min(scores) and round these values to a precision of two decimal places

Then: 
* For each individual example, remove scores below the minRth percentile and greater than maxRth percentile (sort for each example if necessary)
* Call the calc_stats function to compute min_scr and max_scr with the filtered margin scores as parameter
* Return the min_scr and max_scr computed by the **calc_stat** function using the margin scores.
      
**Note**: You can experiment by changing **n_bootstraps**, but it takes about 200 iterations to get consistent values.

In [41]:
def reshape(array, minR, maxR):
    low = np.percentile(array, minR*100)
    high = np.percentile(array, maxR*100)
    
    reshaped = []
    
    for index in range(len(array[0])):
        temp = []
        for i in array:
            if low < i[index] < high:
                temp.append(i[index])
        reshaped.append(temp)
    return reshaped

def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR, maxR, bootstrap_size, \
num_round=100, plst=xgboost_plst()):
    norm_scores = []
    
    for i in range(n_bootstrap):
        
        # Sample bootstrap_size indices from training set w/ replacement
        samp_indices = np.random.randint(len(X_train), size=bootstrap_size)
        
        # Training data and labels
        X_sample = X_train[samp_indices]
        y_sample = y_train[samp_indices]
        
        # Train and test data matrices
        dtrain = xgb.DMatrix(X_sample, label=y_sample)
        dtest = xgb.DMatrix(X_test, label=y_test)
        evallist = [(dtrain, 'train'), (dtest, 'eval')]
        
        # Train model
        bst = xgb.train(plst, dtrain, num_round, evallist)
        pred = bst.predict(dtest, output_margin=True)
        
        # Normalize predictions rounded 2 decimal places
        norm_scores.append(np.sort(np.round(pred/(max(pred)-min(pred)), decimals=2)))
    
    new_scores = reshape(norm_scores, minR, maxR)
    
    # Compute min_scr and max_scr
    stats = calc_stats(new_scores)
    
    return stats



In [42]:
def process(X_train, X_test, y_train, y_test, n_bootstrap=100):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=n_bootstrap, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

#### Tests

How we test the function:
1. We have computed the average mid-point of the range of values and verify that this midpoint is present in the range computed by your function
2. We check that the length of the interval(max_scr-min_scr) is not more than twice the average length of the interval

In [43]:
sample_indices = np.load('Data/vis_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('Data/vis_midpt.npy')
avg_length = np.load('Data/vis_avg_length.npy')
min_scr, max_scr, pred = process(X_train, X_test_samp, y_train, y_test_samp)
length = max_scr - min_scr

[0]	train-logloss:0.55498	eval-logloss:0.598242
[1]	train-logloss:0.479233	eval-logloss:0.566622
[2]	train-logloss:0.43088	eval-logloss:0.552586
[3]	train-logloss:0.397962	eval-logloss:0.546751
[4]	train-logloss:0.372752	eval-logloss:0.522694
[5]	train-logloss:0.352285	eval-logloss:0.534327
[6]	train-logloss:0.336712	eval-logloss:0.556622
[7]	train-logloss:0.325118	eval-logloss:0.516416
[8]	train-logloss:0.314772	eval-logloss:0.54908
[9]	train-logloss:0.30471	eval-logloss:0.586947
[10]	train-logloss:0.294831	eval-logloss:0.582475
[11]	train-logloss:0.286994	eval-logloss:0.589303
[12]	train-logloss:0.280518	eval-logloss:0.563703
[13]	train-logloss:0.276153	eval-logloss:0.588614
[14]	train-logloss:0.270738	eval-logloss:0.572766
[15]	train-logloss:0.263347	eval-logloss:0.56147
[16]	train-logloss:0.259497	eval-logloss:0.573432
[17]	train-logloss:0.254389	eval-logloss:0.55558
[18]	train-logloss:0.250128	eval-logloss:0.544999
[19]	train-logloss:0.246878	eval-logloss:0.534608
[20]	train-loglo

[66]	train-logloss:0.136936	eval-logloss:0.789711
[67]	train-logloss:0.135559	eval-logloss:0.798854
[68]	train-logloss:0.134252	eval-logloss:0.795097
[69]	train-logloss:0.132559	eval-logloss:0.795723
[70]	train-logloss:0.131685	eval-logloss:0.793257
[71]	train-logloss:0.130973	eval-logloss:0.794508
[72]	train-logloss:0.128804	eval-logloss:0.793073
[73]	train-logloss:0.127057	eval-logloss:0.783382
[74]	train-logloss:0.12637	eval-logloss:0.780336
[75]	train-logloss:0.124919	eval-logloss:0.800508
[76]	train-logloss:0.123984	eval-logloss:0.797764
[77]	train-logloss:0.122396	eval-logloss:0.817122
[78]	train-logloss:0.12129	eval-logloss:0.820554
[79]	train-logloss:0.119727	eval-logloss:0.796269
[80]	train-logloss:0.119175	eval-logloss:0.796513
[81]	train-logloss:0.117581	eval-logloss:0.78102
[82]	train-logloss:0.116188	eval-logloss:0.784687
[83]	train-logloss:0.115009	eval-logloss:0.804848
[84]	train-logloss:0.113108	eval-logloss:0.785794
[85]	train-logloss:0.111499	eval-logloss:0.793334
[86

[31]	train-logloss:0.212697	eval-logloss:0.676591
[32]	train-logloss:0.210129	eval-logloss:0.677116
[33]	train-logloss:0.207586	eval-logloss:0.6799
[34]	train-logloss:0.205147	eval-logloss:0.722956
[35]	train-logloss:0.203442	eval-logloss:0.712095
[36]	train-logloss:0.201478	eval-logloss:0.706113
[37]	train-logloss:0.198937	eval-logloss:0.705621
[38]	train-logloss:0.196828	eval-logloss:0.694861
[39]	train-logloss:0.194566	eval-logloss:0.71225
[40]	train-logloss:0.190705	eval-logloss:0.708977
[41]	train-logloss:0.18754	eval-logloss:0.693483
[42]	train-logloss:0.185134	eval-logloss:0.693184
[43]	train-logloss:0.183611	eval-logloss:0.69238
[44]	train-logloss:0.182506	eval-logloss:0.693061
[45]	train-logloss:0.180388	eval-logloss:0.682513
[46]	train-logloss:0.177976	eval-logloss:0.675898
[47]	train-logloss:0.175637	eval-logloss:0.715052
[48]	train-logloss:0.173001	eval-logloss:0.689623
[49]	train-logloss:0.171416	eval-logloss:0.711233
[50]	train-logloss:0.168716	eval-logloss:0.722704
[51]	

[96]	train-logloss:0.104549	eval-logloss:0.551558
[97]	train-logloss:0.103587	eval-logloss:0.551959
[98]	train-logloss:0.102689	eval-logloss:0.56861
[99]	train-logloss:0.101746	eval-logloss:0.562403
[0]	train-logloss:0.553672	eval-logloss:0.598438
[1]	train-logloss:0.475702	eval-logloss:0.589857
[2]	train-logloss:0.427079	eval-logloss:0.575247
[3]	train-logloss:0.393681	eval-logloss:0.581843
[4]	train-logloss:0.3672	eval-logloss:0.573629
[5]	train-logloss:0.348647	eval-logloss:0.575765
[6]	train-logloss:0.334014	eval-logloss:0.578037
[7]	train-logloss:0.321429	eval-logloss:0.584136
[8]	train-logloss:0.31049	eval-logloss:0.592386
[9]	train-logloss:0.302479	eval-logloss:0.613675
[10]	train-logloss:0.29192	eval-logloss:0.593471
[11]	train-logloss:0.284694	eval-logloss:0.62166
[12]	train-logloss:0.276941	eval-logloss:0.640019
[13]	train-logloss:0.272631	eval-logloss:0.639096
[14]	train-logloss:0.267926	eval-logloss:0.649307
[15]	train-logloss:0.263428	eval-logloss:0.670207
[16]	train-loglo

[62]	train-logloss:0.139435	eval-logloss:0.682311
[63]	train-logloss:0.137988	eval-logloss:0.673236
[64]	train-logloss:0.137347	eval-logloss:0.67956
[65]	train-logloss:0.135869	eval-logloss:0.687502
[66]	train-logloss:0.134731	eval-logloss:0.695563
[67]	train-logloss:0.134016	eval-logloss:0.692041
[68]	train-logloss:0.133684	eval-logloss:0.691786
[69]	train-logloss:0.131596	eval-logloss:0.68597
[70]	train-logloss:0.129279	eval-logloss:0.68837
[71]	train-logloss:0.12705	eval-logloss:0.696732
[72]	train-logloss:0.124991	eval-logloss:0.738966
[73]	train-logloss:0.123543	eval-logloss:0.773988
[74]	train-logloss:0.121703	eval-logloss:0.7913
[75]	train-logloss:0.119892	eval-logloss:0.788727
[76]	train-logloss:0.118466	eval-logloss:0.805656
[77]	train-logloss:0.116812	eval-logloss:0.804509
[78]	train-logloss:0.115314	eval-logloss:0.807872
[79]	train-logloss:0.114926	eval-logloss:0.804774
[80]	train-logloss:0.113712	eval-logloss:0.789772
[81]	train-logloss:0.112523	eval-logloss:0.807068
[82]	t

...

[70]	train-logloss:0.139316	eval-logloss:0.685894
[71]	train-logloss:0.137514	eval-logloss:0.675311
[72]	train-logloss:0.135342	eval-logloss:0.673264
[73]	train-logloss:0.133416	eval-logloss:0.673037
[74]	train-logloss:0.132774	eval-logloss:0.683011
[75]	train-logloss:0.132216	eval-logloss:0.671558
[76]	train-logloss:0.131498	eval-logloss:0.673673
[77]	train-logloss:0.130231	eval-logloss:0.676867
[78]	train-logloss:0.128379	eval-logloss:0.712495
[79]	train-logloss:0.127147	eval-logloss:0.743422
[80]	train-logloss:0.12608	eval-logloss:0.748748
[81]	train-logloss:0.124748	eval-logloss:0.74803
[82]	train-logloss:0.124183	eval-logloss:0.748355
[83]	train-logloss:0.12324	eval-logloss:0.73722
[84]	train-logloss:0.121634	eval-logloss:0.740357
[85]	train-logloss:0.121183	eval-logloss:0.76366
[86]	train-logloss:0.12062	eval-logloss:0.763534
[87]	train-logloss:0.118871	eval-logloss:0.768528
[88]	train-logloss:0.118244	eval-logloss:0.768711
[89]	train-logloss:0.117753	eval-logloss:0.782444
[90]	t

[35]	train-logloss:0.206773	eval-logloss:0.520184
[36]	train-logloss:0.203105	eval-logloss:0.512176
[37]	train-logloss:0.199609	eval-logloss:0.541386
[38]	train-logloss:0.198412	eval-logloss:0.531883
[39]	train-logloss:0.197411	eval-logloss:0.530294
[40]	train-logloss:0.195638	eval-logloss:0.528767
[41]	train-logloss:0.193128	eval-logloss:0.523626
[42]	train-logloss:0.190704	eval-logloss:0.521317
[43]	train-logloss:0.187498	eval-logloss:0.534302
[44]	train-logloss:0.18358	eval-logloss:0.556455
[45]	train-logloss:0.181265	eval-logloss:0.55571
[46]	train-logloss:0.178438	eval-logloss:0.549748
[47]	train-logloss:0.1764	eval-logloss:0.546046
[48]	train-logloss:0.174973	eval-logloss:0.544855
[49]	train-logloss:0.173934	eval-logloss:0.53913
[50]	train-logloss:0.170756	eval-logloss:0.536838
[51]	train-logloss:0.168576	eval-logloss:0.541524
[52]	train-logloss:0.166769	eval-logloss:0.578376
[53]	train-logloss:0.165131	eval-logloss:0.564369
[54]	train-logloss:0.161952	eval-logloss:0.560977
[55]	

[0]	train-logloss:0.561941	eval-logloss:0.601502
[1]	train-logloss:0.487955	eval-logloss:0.536453
[2]	train-logloss:0.438355	eval-logloss:0.523841
[3]	train-logloss:0.405934	eval-logloss:0.531117
[4]	train-logloss:0.383206	eval-logloss:0.527421
[5]	train-logloss:0.364195	eval-logloss:0.524589
[6]	train-logloss:0.348289	eval-logloss:0.549717
[7]	train-logloss:0.336897	eval-logloss:0.567301
[8]	train-logloss:0.323809	eval-logloss:0.56136
[9]	train-logloss:0.315759	eval-logloss:0.594748
[10]	train-logloss:0.304862	eval-logloss:0.576152
[11]	train-logloss:0.296325	eval-logloss:0.580456
[12]	train-logloss:0.28979	eval-logloss:0.561759
[13]	train-logloss:0.283769	eval-logloss:0.592563
[14]	train-logloss:0.277345	eval-logloss:0.581768
[15]	train-logloss:0.271774	eval-logloss:0.624404
[16]	train-logloss:0.26582	eval-logloss:0.650307
[17]	train-logloss:0.260465	eval-logloss:0.64908
[18]	train-logloss:0.258599	eval-logloss:0.704605
[19]	train-logloss:0.252541	eval-logloss:0.735908
[20]	train-log

[65]	train-logloss:0.134799	eval-logloss:0.686965
[66]	train-logloss:0.133503	eval-logloss:0.692637
[67]	train-logloss:0.132751	eval-logloss:0.697155
[68]	train-logloss:0.132161	eval-logloss:0.708823
[69]	train-logloss:0.130818	eval-logloss:0.706364
[70]	train-logloss:0.128815	eval-logloss:0.716807
[71]	train-logloss:0.127453	eval-logloss:0.719568
[72]	train-logloss:0.125815	eval-logloss:0.703538
[73]	train-logloss:0.124715	eval-logloss:0.692783
[74]	train-logloss:0.124227	eval-logloss:0.685885
[75]	train-logloss:0.123814	eval-logloss:0.68354
[76]	train-logloss:0.122878	eval-logloss:0.676185
[77]	train-logloss:0.122051	eval-logloss:0.695173
[78]	train-logloss:0.120984	eval-logloss:0.692355
[79]	train-logloss:0.120222	eval-logloss:0.684707
[80]	train-logloss:0.118891	eval-logloss:0.666828
[81]	train-logloss:0.116571	eval-logloss:0.683872
[82]	train-logloss:0.115782	eval-logloss:0.682028
[83]	train-logloss:0.114637	eval-logloss:0.671816
[84]	train-logloss:0.113322	eval-logloss:0.670879
[

[30]	train-logloss:0.208244	eval-logloss:0.669423
[31]	train-logloss:0.206207	eval-logloss:0.699005
[32]	train-logloss:0.203708	eval-logloss:0.703875
[33]	train-logloss:0.202027	eval-logloss:0.70434
[34]	train-logloss:0.199937	eval-logloss:0.725726
[35]	train-logloss:0.199195	eval-logloss:0.727245
[36]	train-logloss:0.196481	eval-logloss:0.710256
[37]	train-logloss:0.193338	eval-logloss:0.710163
[38]	train-logloss:0.191128	eval-logloss:0.718329
[39]	train-logloss:0.187934	eval-logloss:0.723647
[40]	train-logloss:0.185626	eval-logloss:0.736403
[41]	train-logloss:0.182158	eval-logloss:0.725725
[42]	train-logloss:0.179597	eval-logloss:0.724391
[43]	train-logloss:0.178211	eval-logloss:0.729808
[44]	train-logloss:0.175516	eval-logloss:0.739715
[45]	train-logloss:0.173613	eval-logloss:0.746458
[46]	train-logloss:0.170872	eval-logloss:0.752906
[47]	train-logloss:0.16973	eval-logloss:0.75746
[48]	train-logloss:0.16736	eval-logloss:0.765195
[49]	train-logloss:0.16486	eval-logloss:0.757925
[50]	

In [44]:
assert sum(min_scr <= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [45]:
assert sum(max_scr >= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [46]:
assert sum(length < 2*avg_length) >= (0.7 * len(sample_indices)), "Incorrect length of range (mean - 3*std) to (mean + 3*std)"

In [47]:
# Hidden Test Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [48]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [49]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [50]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [51]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [52]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [53]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [54]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [55]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [56]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [57]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#
